<a href="https://colab.research.google.com/github/mcarbel/mygooglecolabpublic/blob/main/ExcelCompare9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Getting started**

The document you are reading is not a static web page, but an interactive environment called a **Colab notebook** that lets you write and execute code.

For example, here is a **code cell** with a short Python script that computes a value, stores it in a variable, and prints the result:

# **Parameters Setup**
# path to files


In [8]:
import pandas as pd
from pathlib import Path

#define parameters
#path to files
path_old=Path(r'Quoted_Borrowings_valuation_ASK_30.06.2021_DEV_EOM_20210630_20210713_Copy.xlsx', sheet_name = 'Detailed', header = 0 , index_col=0)
path_new=Path(r'Quoted_Borrowings_valuation_ASK_30.06.2021_PROD_EOM_20210630_20210713_Copy.xlsx', sheet_name = 'Detailed', header = 0, index_col = 0)



# **Define Index**

In [35]:
#list of key column(s)
key=['Instrument','ISIN','Trn']
#sheets to read in
#sheet='Detailed'

In [69]:
# Read in the two excel files and fill NA
old = pd.read_excel(path_old).fillna(0)
new = pd.read_excel(path_new).fillna(0)


In [70]:
#set index
old=old.set_index(key)
new=new.set_index(key)

In [63]:
old=""

In [64]:
new=""

# **Dropped/Added Rows Dataframe**

In [71]:
#identify dropped rows and added (new) rows
dropped_rows = set(old.index) - set(new.index)
added_rows = set(new.index) - set(old.index)


In [72]:
dropped_rows

set()

In [73]:
added_rows

set()

In [74]:
old

,,,Parent Number,Borr,Instrument Group,Instrument Name,Designation IFRS,IFRS9 Accounting Category Name,Ccy,Trn Nominal Amount,Is Quoted (Valuation date),Trade date,Value date,Maturity date,Sign,Ask Price,Final ZC Repayment (local),Final ZC Repayment (EUR),Trn Figure Accrued Interest (hist),Trn Figure Clean Market Value (hist),Trn Figure Market Value (hist)
Instrument,ISIN,Trn,,,,,,,,,,,,,,,,,,,
GBP-STR-2030-07-22-INFLATION,XS0132108704,169844,169844,2280200,EIB-INFLATION-LINKED,BORR 0228 - 1565/01 - inflation linked - UK-RPI,FVOPL,FVOPL,GBP,-75000000.0,Y,2004-05-12,2004-06-04,2030-07-22,Issue,257.661,0.000000e+00,0.000000e+00,-5.841709e+07,-2.258749e+08,-2.267860e+08
EUR-NST-2026-11-05-ZERO-B,DE0001345908,171091,171091,720200,EIB-ZERO-COUPON,"BORR 0072 - 1418/01 - Zero-coupon, DEM Converted",FVOPL,HEDAC,EUR,-35279139.8,Y,1997-01-07,2005-01-01,2026-11-05,Issue,101.529,-2.556459e+08,-2.556459e+08,-1.194300e+08,-2.595556e+08,-2.595556e+08
GBP-STR-2034-03-31-INFLATION,XS0129477047,171252,171252,2240100,EIB-INFLATION-LINKED,BORR 0224 - DEAL 68 - 1563/01 - inflation link...,MAMCO,MAMCO,GBP,-70000000.0,Y,2001-05-08,2001-05-17,2034-03-31,Issue,245.615,0.000000e+00,0.000000e+00,-6.583542e+07,-1.096807e+08,-1.102429e+08
GBP-STR-2034-09-30-INFLATION,XS0184056520,171568,171568,6470100,EIB-INFLATION-LINKED,"BORR 0647 - 1609/01 - inflation linked UK-RPI,...",MAMCO,MAMCO,GBP,-100000000.0,Y,2004-01-09,2004-01-15,2034-09-30,Issue,225.196,0.000000e+00,0.000000e+00,-5.571730e+07,-1.493109e+08,-1.499257e+08
GBP-STR-2038-03-31-INFLATION,XS0172367921,171569,171569,5380100,EIB-INFLATION-LINKED,"BORR 0538 - 1599/01 - inflation linked UK-RPI,...",MAMCO,MAMCO,GBP,-50000000.0,Y,2003-07-02,2003-07-09,2038-03-31,Issue,246.774,0.000000e+00,0.000000e+00,-2.958682e+07,-9.008768e+07,-9.042917e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EUR-NST-2024-09-13-FIXED-0.875,XS1247736793,1465066,1465066,21751500,EIB-FIXED,BORR 2175 - FIXED-BULLET,0,MAMCO,EUR,-250000000.0,Y,2021-06-24,2021-07-02,2024-09-13,Issue,104.534,0.000000e+00,0.000000e+00,0.000000e+00,-2.613151e+08,-1.301417e+05
USD-NST-2025-06-13-FIXED-2.875,XS1811852109,1465368,1465368,23460200,EIB-FIXED,BORR 2346 FIXED-BULLET,0,HEDAC,USD,-200000000.0,Y,2021-06-25,2021-07-06,2025-06-13,Issue,108.485,0.000000e+00,0.000000e+00,0.000000e+00,-1.824970e+08,-4.034189e+05
CAD-NST-2024-07-30-FIXED-1.750,XS2033281176,1465691,1465691,24030300,EIB-FIXED,BORR 2403 - FIXED-BULLET,0,HEDAC,CAD,-100000000.0,Y,2021-06-25,2021-07-09,2024-07-30,Issue,102.839,0.000000e+00,0.000000e+00,-4.318280e+03,-6.984518e+07,-3.697215e+04


In [75]:
new

,,,Parent Number,Borr,Instrument Group,Instrument Name,Designation IFRS,IFRS9 Accounting Category Name,Ccy,Trn Nominal Amount,Is Quoted (Valuation date),Trade date,Value date,Maturity date,Sign,Ask Price,Final ZC Repayment (local),Final ZC Repayment (EUR),Trn Figure Accrued Interest (hist),Trn Figure Clean Market Value (hist),Trn Figure Market Value (hist)
Instrument,ISIN,Trn,,,,,,,,,,,,,,,,,,,
GBP-STR-2030-07-22-INFLATION,XS0132108704,169844,169844,2280200,EIB-INFLATION-LINKED,BORR 0228 - 1565/01 - inflation linked - UK-RPI,FVOPL,FVOPL,GBP,-75000000.0,Y,2004-05-12,2004-06-04,2030-07-22,Issue,257.661,0.000000e+00,0.000000e+00,-5.841709e+07,-2.258749e+08,-2.267860e+08
EUR-NST-2026-11-05-ZERO-B,DE0001345908,171091,171091,720200,EIB-ZERO-COUPON,"BORR 0072 - 1418/01 - Zero-coupon, DEM Converted",FVOPL,HEDAC,EUR,-35279139.8,Y,1997-01-07,2005-01-01,2026-11-05,Issue,101.529,-2.556459e+08,-2.556459e+08,-1.194300e+08,-2.595556e+08,-2.595556e+08
GBP-STR-2034-03-31-INFLATION,XS0129477047,171252,171252,2240100,EIB-INFLATION-LINKED,BORR 0224 - DEAL 68 - 1563/01 - inflation link...,MAMCO,MAMCO,GBP,-70000000.0,Y,2001-05-08,2001-05-17,2034-03-31,Issue,245.615,0.000000e+00,0.000000e+00,-6.583542e+07,-1.096807e+08,-1.102429e+08
GBP-STR-2034-09-30-INFLATION,XS0184056520,171568,171568,6470100,EIB-INFLATION-LINKED,"BORR 0647 - 1609/01 - inflation linked UK-RPI,...",MAMCO,MAMCO,GBP,-100000000.0,Y,2004-01-09,2004-01-15,2034-09-30,Issue,225.196,0.000000e+00,0.000000e+00,-5.571730e+07,-1.493109e+08,-1.499257e+08
GBP-STR-2038-03-31-INFLATION,XS0172367921,171569,171569,5380100,EIB-INFLATION-LINKED,"BORR 0538 - 1599/01 - inflation linked UK-RPI,...",MAMCO,MAMCO,GBP,-50000000.0,Y,2003-07-02,2003-07-09,2038-03-31,Issue,246.774,0.000000e+00,0.000000e+00,-2.958682e+07,-9.008768e+07,-9.042917e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EUR-NST-2024-09-13-FIXED-0.875,XS1247736793,1465066,1465066,21751500,EIB-FIXED,BORR 2175 - FIXED-BULLET,0,MAMCO,EUR,-250000000.0,Y,2021-06-24,2021-07-02,2024-09-13,Issue,104.534,0.000000e+00,0.000000e+00,0.000000e+00,-2.613151e+08,-1.301417e+05
USD-NST-2025-06-13-FIXED-2.875,XS1811852109,1465368,1465368,23460200,EIB-FIXED,BORR 2346 FIXED-BULLET,0,HEDAC,USD,-200000000.0,Y,2021-06-25,2021-07-06,2025-06-13,Issue,108.485,0.000000e+00,0.000000e+00,0.000000e+00,-1.824970e+08,-4.034189e+05
CAD-NST-2024-07-30-FIXED-1.750,XS2033281176,1465691,1465691,24030300,EIB-FIXED,BORR 2403 - FIXED-BULLET,0,HEDAC,CAD,-100000000.0,Y,2021-06-25,2021-07-09,2024-07-30,Issue,102.839,0.000000e+00,0.000000e+00,-4.318280e+03,-6.984518e+07,-3.697215e+04


# **Combine data for comparison**

In [76]:
#combine data
df_all_changes = pd.concat([old, new], axis='columns', keys=['old','new'], join='inner')

In [77]:


#prepare function for comparing old values and new values
def report_diff(x):
    return x[0] if x[0] == x[1] else '{} ---> {}'.format(*x)





In [78]:
#swap column indexes
df_all_changes = df_all_changes.swaplevel(axis='columns')[new.columns[0:]]

In [79]:
#apply the report_diff function
df_changed = df_all_changes.groupby(level=0, axis=1).apply(lambda frame: frame.apply(report_diff, axis=1))

In [80]:
#create a list of text columns (int columns do not have '{} ---> {}')
df_changed_text_columns = df_changed.select_dtypes(include='object')


In [81]:
#create 3 datasets:
#diff - contains the differences
#dropped - contains the dropped rows
#added - contains the added rows
diff = df_changed_text_columns[df_changed_text_columns.apply(lambda x: x.str.contains("--->") == True, axis=1)]
dropped = old.loc[dropped_rows]
added = new.loc[added_rows]


#create a name for the output excel file
fname =  '{} vs {}.xlsx'.format(path_old.stem, path_new.stem)



In [85]:
diff

,,,Ccy,Designation IFRS,Final ZC Repayment (EUR),IFRS9 Accounting Category Name,Instrument Group,Instrument Name,Is Quoted (Valuation date),Sign
Instrument,ISIN,Trn,,,,,,,,
GBP-STR-2030-07-22-INFLATION,XS0132108704,169844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EUR-NST-2026-11-05-ZERO-B,DE0001345908,171091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBP-STR-2034-03-31-INFLATION,XS0129477047,171252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBP-STR-2034-09-30-INFLATION,XS0184056520,171568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBP-STR-2038-03-31-INFLATION,XS0172367921,171569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
EUR-NST-2024-09-13-FIXED-0.875,XS1247736793,1465066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USD-NST-2025-06-13-FIXED-2.875,XS1811852109,1465368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAD-NST-2024-07-30-FIXED-1.750,XS2033281176,1465691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Write dataframe to excel

In [86]:
pip install XlsxWriter

     |████████████████████████████████| 149 kB 26.6 MB/s 


In [90]:
#write dataframe to excel
writer=pd.ExcelWriter(fname, engine='xlsxwriter')
diff.to_excel(writer, sheet_name='diff', index=True)
dropped.to_excel(writer, sheet_name='dropped', index=True)
added.to_excel(writer, sheet_name='added', index=True)

IndexError: ignored

In [91]:
#get xlswriter objects
workbook = writer.book
worksheet = writer.sheets['diff']
worksheet.hide_gridlines(2)
worksheet.set_default_row(15)

In [92]:
#get number of rows of the df diff
row_count_str=str(len(diff.index)+1)

In [93]:
#define and apply formats
highligt_fmt = workbook.add_format({'font_color': '#FF0000', 'bg_color':'#B1B3B3'})
worksheet.conditional_format('A1:ZZ'+row_count_str, {'type':'text', 'criteria':'containing', 'value':'--->',
                            'format':highligt_fmt})


0

In [94]:
#save the output
writer.save()
print ('\nDone.\n')


Done.

